In [1]:
# Data stuff
import numpy as np
import pandas as pd

# Visual stuff
from IPython.display import display
# Configs 
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import matplotlib.pyplot as plt

# Random state seed
rand_state=42

# ML stuff
import sklearn
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
import shap

# Imbalanced learn 
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTENC

In [2]:
dataset = pd.read_csv('data_gen/dataset_mean_entire.csv')
# Ensure correct types
dataset = dataset.astype(dtype={
    'age':float,
    'gender':int,
    'expired':int,
    'P-glucose':float,
    'blood_pressure_systoliskt':float,
    'blood_pressure_diastoliskt':float,
    'BMI':float
    
})
# Order ints(categorical variables, first) and floats(number variables, last)
dataset = dataset[[
    'gender',
    'I109',
    'E119',
    'E669',
    'I259',
    'I252',
    'I209',
    'E660',
    'E118',
    'I639',
    'E113',
    'expired',
    'age',
    'P-glucose',
    'blood_pressure_systoliskt',
    'blood_pressure_diastoliskt',
    'BMI',
]]

In [3]:
# Generate data set withouth categories(all numbers)
Y_no_cat = dataset.expired.values
X_no_cat = dataset.drop(columns=['expired']).values

# dataset as is, but target variable dropped(hospital expire flag)
dataset_no_target = dataset.drop(columns=['expired'])

# Feture names and categorical feature names
feature_names = dataset_no_target.select_dtypes(include='int').columns.values.tolist() + dataset_no_target.select_dtypes(exclude='int').columns.values.tolist() 
cat_feature_names = dataset_no_target.select_dtypes(include='int').columns.values.tolist() 

# Generate data set with categories(int type required)
dataframe_int_list = dataset_no_target.select_dtypes(include='int').values.tolist()
dataframe_no_int_list = dataset_no_target.select_dtypes(exclude='int').values.tolist()
Y = dataset.expired.values.tolist()
X = []
for i,v in enumerate(dataframe_int_list):
    X = X + [v+dataframe_no_int_list[i]]

# Generate categorical feature indicies
cat_features_indices=list(range(0,len(dataframe_int_list[0])))

In [4]:
def strat_cv_it(classifier, params, uses_cat, param_comb ):
    folds = 3
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = rand_state)
    
    imba_pipeline = make_pipeline(SMOTENC(random_state=42,categorical_features=cat_features_indices), 
                                 classifier)
    
    # Perform fit and scoring
    if uses_cat:
        random_search = RandomizedSearchCV(imba_pipeline, return_train_score=True, param_distributions=params ,scoring='roc_auc', n_iter=param_comb, n_jobs=4, cv=skf.split(X, Y),random_state = rand_state, refit=True, verbose=3,  )
        random_search.fit(X, Y)     
    else:
        random_search = RandomizedSearchCV(imba_pipeline, return_train_score=True, param_distributions=params ,scoring='roc_auc', n_iter=param_comb, n_jobs=4, cv=skf.split(X_no_cat, Y_no_cat),random_state = rand_state, refit=True, verbose=3)
        random_search.fit(X_no_cat, Y_no_cat)
    
    # Display results and return best model
    display(random_search.best_score_)
    display(random_search.best_params_)
    display(pd.DataFrame(random_search.cv_results_))
    return random_search.best_estimator_

# XGboost

In [5]:
xgb_classifier = xgb.XGBClassifier(objective = "binary:logistic",random_state=rand_state)
xgb_params = {
        'xgbclassifier__learning_rate': (0.01, 0.05,0.1),
        'xgbclassifier__min_child_weight': [3, 5, 10],
        'xgbclassifier__gamma': [0.5, 1.5, 2, 5],
        'xgbclassifier__subsample': [0.6, 0.8, 1.0],
        'xgbclassifier__colsample_bytree': [0.6, 0.8, 1.0],
        'xgbclassifier__max_depth': list(range(5,30)),
        'xgbclassifier__scale_pos_weight':  [0.5,0.75,0.9,1, 1.1, 1.25,1.5]
}
model = strat_cv_it(xgb_classifier ,xgb_params,False,50)
explainer = shap.TreeExplainer(model['xgbclassifier'])

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  5.4min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  9.8min
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed: 10.2min finished


0.8747475811774414

{'xgbclassifier__subsample': 0.6,
 'xgbclassifier__scale_pos_weight': 0.5,
 'xgbclassifier__min_child_weight': 10,
 'xgbclassifier__max_depth': 5,
 'xgbclassifier__learning_rate': 0.01,
 'xgbclassifier__gamma': 0.5,
 'xgbclassifier__colsample_bytree': 0.8}

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_xgbclassifier__subsample,param_xgbclassifier__scale_pos_weight,param_xgbclassifier__min_child_weight,param_xgbclassifier__max_depth,param_xgbclassifier__learning_rate,param_xgbclassifier__gamma,param_xgbclassifier__colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,52.796271,0.905573,0.048568,0.005202,0.8,1.25,5,25,0.1,5,1,"{'xgbclassifier__subsample': 0.8, 'xgbclassifi...",0.858449,0.845841,0.834889,0.846393,0.009626,44,0.948329,0.949588,0.953024,0.950314,0.001984
1,17.374547,6.473804,0.024163,0.009604,0.6,0.75,10,5,0.05,5,0.6,"{'xgbclassifier__subsample': 0.6, 'xgbclassifi...",0.885710,0.863838,0.855215,0.868254,0.012835,2,0.893025,0.902034,0.899455,0.898171,0.003788
2,65.119902,5.164504,0.060256,0.006358,1,1.5,5,18,0.01,0.5,0.6,"{'xgbclassifier__subsample': 1.0, 'xgbclassifi...",0.874751,0.857075,0.845913,0.859246,0.011872,20,0.935572,0.937266,0.938197,0.937012,0.001087
3,52.008579,5.429532,0.046212,0.002739,0.8,0.5,10,10,0.01,0.5,1,"{'xgbclassifier__subsample': 0.8, 'xgbclassifi...",0.878767,0.866127,0.849415,0.864770,0.012022,7,0.900963,0.907300,0.898550,0.902271,0.003690
4,85.088382,3.563305,0.042879,0.006056,0.8,1.25,5,17,0.05,5,1,"{'xgbclassifier__subsample': 0.8, 'xgbclassifi...",0.870461,0.853198,0.841416,0.855025,0.011928,28,0.938399,0.937718,0.942466,0.939528,0.002096
5,66.314384,11.632509,0.037405,0.004759,1,0.5,3,15,0.05,1.5,1,"{'xgbclassifier__subsample': 1.0, 'xgbclassifi...",0.865836,0.853297,0.839434,0.852856,0.010783,36,0.950713,0.953575,0.956976,0.953755,0.002560
6,23.212505,2.835464,0.036503,0.008279,0.8,0.9,3,9,0.05,2,0.6,"{'xgbclassifier__subsample': 0.8, 'xgbclassifi...",0.874660,0.849217,0.840269,0.854715,0.014568,32,0.934024,0.936456,0.939449,0.936643,0.002219
7,49.722251,2.939617,0.039614,0.002688,0.8,1.1,3,26,0.05,5,1,"{'xgbclassifier__subsample': 0.8, 'xgbclassifi...",0.865933,0.854682,0.841738,0.854117,0.009886,33,0.946624,0.947985,0.950084,0.948231,0.001423
8,30.796045,2.964678,0.034494,0.015754,0.8,0.9,5,29,0.01,1.5,0.6,"{'xgbclassifier__subsample': 0.8, 'xgbclassifi...",0.879626,0.853016,0.853235,0.861959,0.012492,11,0.917779,0.922275,0.925200,0.921751,0.003052
9,34.692785,0.470997,0.035709,0.005575,1,0.9,5,22,0.05,5,0.6,"{'xgbclassifier__subsample': 1.0, 'xgbclassifi...",0.870770,0.854010,0.844463,0.856414,0.010873,24,0.933456,0.936086,0.937513,0.935685,0.001681


In [ ]:
shap_values = explainer.shap_values( X_no_cat)
shap.summary_plot(shap_values,X_no_cat ,feature_names=feature_names, show=False)
#plt.savefig("shap_summary.svg", format='svg', dpi=300, bbox_inches='tight')
for predictor in feature_names:
    if predictor != 'age':
        save = shap.dependence_plot(predictor, shap_values,X_no_cat ,feature_names=feature_names,interaction_index='age', show=False)
    else:
        save = shap.dependence_plot(predictor, shap_values,X_no_cat ,feature_names=feature_names, interaction_index='gender', show=False)
        
    # plt.savefig(predictor+".svg", format='svg', dpi=300, bbox_inches='tight')

# LightGBM

In [7]:
LGB_classifier = lgb.LGBMClassifier()
LGB_params = {
             'lgbmclassifier__num_leaves': [1,5,8,10,15,20,35,40], 
             'lgbmclassifier__min_child_samples': [1,5,10,20,50,100,200,300,400,500], 
             'lgbmclassifier__min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'lgbmclassifier__subsample': [0.2,0,4,0.5, 0.6, 0.8, 1.0],
             'lgbmclassifier__colsample_bytree': [0.6, 0.8, 1.0],
             'lgbmclassifier__reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'lgbmclassifier__reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
             'lgbmclassifier__scale_pos_weight':  [0.5,0.75,0.9,1, 1.1, 1.25,1.5]

}
model = strat_cv_it(LGB_classifier ,LGB_params,True,50)
explainer = shap.TreeExplainer(model['lgbmclassifier'])

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    5.1s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:   20.2s
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed:   24.3s finished


0.8828351657163713

{'lgbmclassifier__subsample': 0.6,
 'lgbmclassifier__scale_pos_weight': 0.9,
 'lgbmclassifier__reg_lambda': 100,
 'lgbmclassifier__reg_alpha': 100,
 'lgbmclassifier__num_leaves': 10,
 'lgbmclassifier__min_child_weight': 10.0,
 'lgbmclassifier__min_child_samples': 500,
 'lgbmclassifier__colsample_bytree': 0.8}

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lgbmclassifier__subsample,param_lgbmclassifier__scale_pos_weight,param_lgbmclassifier__reg_lambda,param_lgbmclassifier__reg_alpha,param_lgbmclassifier__num_leaves,param_lgbmclassifier__min_child_weight,param_lgbmclassifier__min_child_samples,param_lgbmclassifier__colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.562849,0.024771,0.010314,0.003508,0.2,0.5,50,50,5,0.01,100,1,"{'lgbmclassifier__subsample': 0.2, 'lgbmclassi...",0.892297,0.876357,0.867850,0.878835,0.010133,2,0.889193,0.896775,0.889753,0.891907,0.003450
1,0.590230,0.021053,0.015086,0.005910,0.8,0.5,50,5,20,0.1,300,1,"{'lgbmclassifier__subsample': 0.8, 'lgbmclassi...",0.887424,0.862557,0.851413,0.867131,0.015053,15,0.891923,0.897199,0.899631,0.896251,0.003217
2,0.967347,0.107294,0.016852,0.000736,1,1,0.1,10,15,0.01,100,1,"{'lgbmclassifier__subsample': 1.0, 'lgbmclassi...",0.873021,0.860115,0.846971,0.860036,0.010635,19,0.901735,0.909712,0.909118,0.906855,0.003628
3,0.423968,0.008810,0.000000,0.000000,1,0.9,50,10,1,1000,100,1,"{'lgbmclassifier__subsample': 1.0, 'lgbmclassi...",NaN,NaN,NaN,NaN,NaN,47,NaN,NaN,NaN,NaN,NaN
4,0.618262,0.055327,0.017097,0.001426,0.5,1.25,0.1,50,40,10,200,0.6,"{'lgbmclassifier__subsample': 0.5, 'lgbmclassi...",0.886930,0.871653,0.867383,0.875322,0.008391,7,0.886320,0.891717,0.887304,0.888447,0.002347
5,0.445853,0.017314,0.011635,0.003497,1,0.9,0,0.1,15,10000,200,0.8,"{'lgbmclassifier__subsample': 1.0, 'lgbmclassi...",0.500000,0.500000,0.500000,0.500000,0.000000,23,0.500000,0.500000,0.500000,0.500000,0.000000
6,0.627007,0.073103,0.013340,0.003854,0.5,1,10,2,15,0.1,500,0.8,"{'lgbmclassifier__subsample': 0.5, 'lgbmclassi...",0.886505,0.864686,0.856642,0.869277,0.012616,12,0.890725,0.897718,0.899877,0.896107,0.003906
7,0.403298,0.003733,0.000000,0.000000,1,1.1,10,0,1,0.001,500,1,"{'lgbmclassifier__subsample': 1.0, 'lgbmclassi...",NaN,NaN,NaN,NaN,NaN,31,NaN,NaN,NaN,NaN,NaN
8,0.570132,0.090120,0.017793,0.005517,0.8,1.25,0.1,2,5,1000,400,0.6,"{'lgbmclassifier__subsample': 0.8, 'lgbmclassi...",0.500000,0.500000,0.500000,0.500000,0.000000,23,0.500000,0.500000,0.500000,0.500000,0.000000
9,0.717813,0.072621,0.009111,0.001760,1,1.5,1,10,8,100,400,1,"{'lgbmclassifier__subsample': 1.0, 'lgbmclassi...",0.886902,0.863366,0.857075,0.869114,0.012837,13,0.886185,0.893119,0.892725,0.890676,0.003180


In [ ]:
shap_values = explainer.shap_values(dataset.drop(columns=['expired']))
shap.summary_plot(shap_values[1],dataset.drop(columns=['expired']) ,feature_names=feature_names, show=False)
#plt.savefig("shap_summary.svg", format='svg', dpi=300, bbox_inches='tight')
for predictor in feature_names:
    if predictor != 'age':
        save = shap.dependence_plot(predictor, shap_values[1],dataset.drop(columns=['expired']) ,feature_names=feature_names,interaction_index='age', show=False)
    else:
        save = shap.dependence_plot(predictor, shap_values[1],dataset.drop(columns=['expired']) ,feature_names=feature_names, interaction_index='gender', show=False)
        
  #  plt.savefig(predictor+".svg", format='svg', dpi=300, bbox_inches='tight')